In [96]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated, Literal
from langchain_core.messages import BaseMessage
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from datetime import datetime
import uuid

load_dotenv()

True

In [97]:
model = ChatGroq(model="llama-3.1-8b-instant")

In [98]:
class SessionState(BaseModel):
    """Metadata for session tracking"""
    session_id: str = f"{datetime.now().strftime('%Y%m%d')}_{datetime.now().strftime('%H%M%S')}"
    session_focus:str = Field(description="Brief description of session focus")
    dominant_psychological_state: Literal["child_self", "adult_self", "mixed"] = Field(default="mixed")
    crisis_level: Literal["low", "medium", "high"] = Field(default="low")
    session_stage: Literal["opening", "exploration", "intervention", "processing", "closing"] = Field(default="opening")
    therapeutic_alliance: Literal["strong", "developing", "weak"] = Field(default="developing")

class ClientProfile(BaseModel):
    """Client information and psychological profile"""
    client_id:str = Field(description="Unique identifier for the client")
    client_name:str = Field(description="Name of the client")
    client_age:int = Field(default=0,description="Age of the client")
    primary_concerns: list[str] = Field(default_factory=list, description="Main therapeutic concerns")
    trauma_history: list[str] = Field(default_factory=list, description="Known trauma history")
    current_medications: list[str] = Field(default_factory=list, description="Current medications")
    previous_therapy_experience: bool = Field(default=False, description="Has previous therapy experience")

class PsychologicalAssessment(BaseModel):
    """Current psychological state assessment"""
    emotional_intensity: int = Field(default=5, ge=1, le=10, description="Emotional intensity rating 1-10")
    emotional_stability: Literal["stable", "fluctuating", "volatile"] = Field(default="stable")
    self_criticism_level: Literal["low", "medium", "high"] = Field(default="medium")
    self_compassion_level: Literal["low", "medium", "high"] = Field(default="medium")
    adult_self_rating: int = Field(default=5, ge=1, le=10, description="Adult self functioning rating")
    child_self_rating: int = Field(default=5, ge=1, le=10, description="Child self activation rating")

class TherapeuticOpportunities(BaseModel):
    """Identified opportunities for therapeutic intervention"""
    perspective_shifting_moments: list[str] = Field(default_factory=list)
    compassion_building_opportunities: list[str] = Field(default_factory=list)
    adult_self_activation_points: list[str] = Field(default_factory=list)
    trauma_processing_areas: list[str] = Field(default_factory=list)

class SessionRecommendations(BaseModel):
    """Recommendations for current and future sessions"""
    immediate_interventions: list[str] = Field(default_factory=list)
    homework_suggestions: list[str] = Field(default_factory=list)
    follow_up_areas: list[str] = Field(default_factory=list)
    crisis_monitoring: list[str] = Field(default_factory=list)

class AnalysisResults(BaseModel):
    """Results of the session analysis"""
    named_entities: dict = Field(default_factory=dict, description="Extracted people, places, events, etc.")
    emotional_state: dict = Field(default_factory=dict, description="Current emotional assessment")
    trauma_indicators: list[dict] = Field(default_factory=list, description="Identified trauma markers")
    coping_mechanisms: dict = Field(default_factory=dict, description="Healthy and unhealthy coping strategies")
    relationship_patterns: list[dict] = Field(default_factory=list, description="Identified relationship patterns")
    progress_indicators: dict = Field(default_factory=dict, description="Signs of progress or concern")

### State

In [99]:
class State(TypedDict):
    """State for the session graph"""
    messages: Annotated[list[BaseMessage], add_messages]
    session_metadata: SessionState
    client_profile: ClientProfile
    psychological_assessment: PsychologicalAssessment
    therapeutic_opportunities: TherapeuticOpportunities
    session_recommendations: SessionRecommendations
    analysis_results: AnalysisResults

### Structured Output

In [100]:
class PersonEntity(BaseModel):
    """Named person in the client's narrative"""
    name: str = Field(description="Name of the person mentioned")
    relationship: str = Field(description="Relationship to the client (e.g., 'mother', 'ex-partner', 'friend')")
    significance: str = Field(description="Why this person is significant in the therapeutic context")
    emotional_charge: Literal["positive", "negative", "neutral"] = Field(description="Emotional association the client has with this person")

class PlaceEntity(BaseModel):
    """Named place in the client's narrative"""
    location: str = Field(description="Name or description of the place")
    context: str = Field(description="Context in which this place was mentioned")
    emotional_association: str = Field(description="Emotional significance of this place to the client")

class EventEntity(BaseModel):
    """Significant event in the client's narrative"""
    event: str = Field(description="Description of the event")
    timeframe: str = Field(description="When this event occurred (e.g., 'childhood', 'last year', 'recently')")
    trauma_relevance: Literal["high", "medium", "low"] = Field(description="How relevant this event is to trauma processing")
    description: str = Field(description="Additional therapeutic context about this event")

class SubstanceEntity(BaseModel):
    """Substance use mentioned by client"""
    substance: str = Field(description="Type of substance (alcohol, drugs, medication, etc.)")
    usage_pattern: str = Field(description="How the substance is being used")
    context: str = Field(description="Context around substance use - coping mechanism, medical, social, etc.")

class NamedEntities(BaseModel):
    """Collection of named entities extracted from client response"""
    people: list[PersonEntity] = Field(default_factory=list, description="People mentioned by the client")
    places: list[PlaceEntity] = Field(default_factory=list, description="Places mentioned by the client")
    events: list[EventEntity] = Field(default_factory=list, description="Significant events mentioned by the client")
    substances: list[SubstanceEntity] = Field(default_factory=list, description="Substances mentioned by the client")

class EmotionalState(BaseModel):
    """Current emotional state assessment"""
    primary_emotions: list[str] = Field(description="Main emotions the client is experiencing")
    emotional_intensity: int = Field(ge=1, le=10, description="Intensity of emotions on a scale of 1-10")
    emotional_stability: Literal["stable", "fluctuating", "volatile"] = Field(description="How stable the client's emotions appear")

class TraumaIndicator(BaseModel):
    """Identified trauma indicator"""
    indicator: str = Field(description="Specific trauma indicator observed")
    severity: Literal["mild", "moderate", "severe"] = Field(description="Severity level of this trauma indicator")
    origin: Literal["childhood", "adult", "unknown"] = Field(description="Likely origin period of this trauma")

class CopingMechanisms(BaseModel):
    """Identified coping strategies"""
    healthy: list[str] = Field(default_factory=list, description="Healthy coping mechanisms the client uses")
    unhealthy: list[str] = Field(default_factory=list, description="Unhealthy but not necessarily harmful coping mechanisms")
    maladaptive: list[str] = Field(default_factory=list, description="Maladaptive or harmful coping mechanisms")

class SelfPerception(BaseModel):
    """Client's self-perception assessment"""
    adult_self_rating: int | None = Field(default=None, ge=1, le=10, description="How well the adult self is functioning (1-10 scale)")
    child_self_rating: int | None = Field(default=None, ge=1, le=10, description="How activated/distressed the child self appears (1-10 scale)")
    self_criticism_level: Literal["low", "medium", "high"] = Field(description="Level of self-criticism observed")
    self_compassion_level: Literal["low", "medium", "high"] = Field(description="Level of self-compassion observed")

class RelationshipPattern(BaseModel):
    """Identified relationship pattern"""
    pattern: str = Field(description="Description of the relationship pattern")
    frequency: str = Field(description="How often this pattern occurs")
    trauma_connection: str = Field(description="How this pattern connects to trauma or childhood experiences")

class TherapeuticOpportunityAnalysis(BaseModel):
    """Identified therapeutic opportunities"""
    perspective_shifting_moments: list[str] = Field(default_factory=list, description="Moments where perspective shifting could be helpful")
    compassion_building_opportunities: list[str] = Field(default_factory=list, description="Opportunities to build self-compassion")
    adult_self_activation_points: list[str] = Field(default_factory=list, description="Points where the adult self could be activated")
    trauma_processing_areas: list[str] = Field(default_factory=list, description="Areas that need trauma processing work")

class SessionRecommendationAnalysis(BaseModel):
    """Session recommendations from analysis"""
    immediate_interventions: list[str] = Field(default_factory=list, description="Interventions needed immediately in this session")
    homework_suggestions: list[str] = Field(default_factory=list, description="Suggested homework or practice for between sessions")
    follow_up_areas: list[str] = Field(default_factory=list, description="Areas to follow up on in future sessions")
    crisis_monitoring: list[str] = Field(default_factory=list, description="Areas that need crisis monitoring")

class ProgressIndicators(BaseModel):
    """Signs of progress or concern"""
    positive_signs: list[str] = Field(default_factory=list, description="Positive signs of therapeutic progress")
    concerning_patterns: list[str] = Field(default_factory=list, description="Patterns that are concerning or need attention")
    therapeutic_alliance: Literal["strong", "developing", "weak"] = Field(description="Strength of the therapeutic relationship")
    insight_level: Literal["high", "medium", "low"] = Field(description="Client's level of psychological insight")

class SessionMetadataAnalysis(BaseModel):
    """Session metadata from analysis"""
    timestamp: str = Field(description="Current timestamp")
    session_focus: str = Field(description="Main focus or theme of this session")
    dominant_psychological_state: Literal["child_self", "adult_self", "mixed"] = Field(description="Which psychological state is most dominant")
    crisis_level: Literal["low", "medium", "high"] = Field(description="Level of crisis or immediate concern")

class PsychologicalAssessmentAnalysis(BaseModel):
    """Comprehensive psychological assessment"""
    emotional_state: EmotionalState = Field(description="Current emotional state assessment")
    trauma_indicators: list[TraumaIndicator] = Field(default_factory=list, description="Identified trauma indicators")
    coping_mechanisms: CopingMechanisms = Field(description="Assessment of coping strategies")
    self_perception: SelfPerception = Field(description="Client's self-perception assessment")
    relationship_patterns: list[RelationshipPattern] = Field(default_factory=list, description="Identified relationship patterns")

class DualBrainAnalysis(BaseModel):
    """Complete Dual Brain Psychology analysis of client response"""
    session_metadata: SessionMetadataAnalysis = Field(description="Session metadata and context")
    named_entities: NamedEntities = Field(description="Named entities extracted from client response")
    psychological_assessment: PsychologicalAssessmentAnalysis = Field(description="Comprehensive psychological assessment")
    therapeutic_opportunities: TherapeuticOpportunityAnalysis = Field(description="Identified therapeutic opportunities")
    session_recommendations: SessionRecommendationAnalysis = Field(description="Recommendations for current and future sessions")
    progress_indicators: ProgressIndicators = Field(description="Signs of progress or areas of concern")

### Initial state

In [101]:
def create_initial_state(client_id: str = "default_client", client_name: str = "Unknown") -> State:
    """
    Create an initial state for a new therapy session.
    """
    
    return State(
        messages=[],
        session_metadata=SessionState(
            session_focus="Initial assessment",
            dominant_psychological_state="mixed",
            crisis_level="low",
            session_stage="opening"
        ),
        client_profile=ClientProfile(
            client_id=client_id,
            client_name=client_name
        ),
        psychological_assessment=PsychologicalAssessment(),
        therapeutic_opportunities=TherapeuticOpportunities(),
        session_recommendations=SessionRecommendations(),
        analysis_results=AnalysisResults()
    )

In [102]:
state=SessionState(session_focus="Initial session focus")

In [103]:
state.session_stage

'opening'